In [ ]:
# OVERVIEW

# Background: 
# The 1st Round of the NFL Draft is a phenomenon that can make or break a team's next season.
# We will experiment with the theory that if a team did well on Offense and poorly on Defense in the previous season, 
# then they would use their highest pick on a player that can improve their Defense (or vice-versa)

# Goal: Predict whether or not a team will draft an Offensive or Defensive player in the 1st Round of the NFL Draft

# Data Points:
# Independent Variable: What unit for the position each team drafted in the 1st Round (Offense=OFF or Defense=DEF)
# (teams can have multiple 1st round picks)
# Dependent Variable: A team's stats and league-wide rankings for the previous season

# Example:
# Bengals 2020 Draft QB=Offense=OFF - Bengals 2019 total passing yards, total rushing yards, turnovers, points for/against, etc.
# Bengals 2019 Stats - https://www.pro-football-reference.com/teams/cin/2019.htm

# Data Scraped From:
# 1st Round Draft History Data - http://www.drafthistory.com/index.php/rounds/round_1
# Team Stats and Rankings - https://www.pro-football-reference.com

In [1]:
# IMPORTS
# DataFrame
import pandas as pd
import numpy as np

# Web Scraping
from requests import get
from bs4 import BeautifulSoup

# ML Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# Math
import math

In [2]:
# CONSTANTS
CURRENT_YEAR = 2020
YEARS = 20
NUM_NEIGHBORS = 5
TEAM_DICT = {'Bengals':'cin','Redskins':'was','Lions':'det','Giants':'nyg','Dolphins':'mia','Chargers':'sdg',
            'Panthers':'car','Cardinals':'crd','Jaguars':'jax','Browns':'cle','Jets':'nyj','Raiders':'rai',
             'Buccaneers':'tam','49ers':'sfo','Broncos':'den','Falcons':'atl','Cowboys':'dal','Eagles':'phi',
             'Vikings':'min','Saints':'nor','Packers':'gnb','Seahawks':'sea','Ravens':'rav','Titans':'oti',
             'Chiefs':'kan','Colts':'clt','Texans':'htx','Bears':'chi','Steelers':'pit','Rams':'ram','Bills':'buf',
             'Patriots':'nwe'}
EMPTY_COLS = ['Off. Rank Ply', 'Off. Rank Y/P', 'Off. Rank Cmp', 'Off. Rank 1stD', 'Off. Rank 1stD', 'Off. Rank Pen',
             'Off. Rank Yds', 'Off. Rank 1stPy', 'Off. Rank #Dr', 'Def. Rank Ply', 'Def. Rank Y/P', 'Def. Rank Cmp',
             'Def. Rank 1stD', 'Def. Rank 1stD', 'Def. Rank Pen', 'Def. Rank Yds', 'Def. Rank 1stPy', 'Def. Rank #Dr',
             'Off. Rank 1stD.1', 'Off. Rank 1stD.2', 'Off. Rank Yds.3', 'Def. Rank 1stD.1', 'Def. Rank 1stD.2', 'Def. Rank Yds.3']
UNIT_DICT = {'QB':1,'RB':1,'WR':1,'TE':1,' C':1,' G':1,' T':1,'DT':2,'DE':2,'LB':2,'DB':2}
POS_DICT = {'QB':1,'RB':2,'WR':3,'TE':4,' C':5,' G':6,' T':7,'DT':8,'DE':9,'LB':10,'DB':11}
BASIC_FACTORS = ['Position Drafted', 'Team', 'Draft Year', 'Pick Number']
TEAM_FACTORS = ['PF', 'Yds', 'Ply', 'Y/P', 'TO', 'FL', '1stD', 'Cmp', 'Att', 'Yds.1', 'TD', 'Int', 'NY/A', '1stD.1', 'Att.1', 'Yds.2', 'TD.1', 'Y/A', '1stD.2', 'Pen', 'Yds.3', '1stPy', '#Dr', 'Sc%', 'TO%', 'Start', 'Time', 'Plays', 'Yds.4', 'Pts']
OPP_FACTORS = ['PF', 'Yds', 'Ply', 'Y/P', 'TO', 'FL', '1stD', 'Cmp', 'Att', 'Yds.1', 'TD', 'Int', 'NY/A', '1stD.1', 'Att.1', 'Yds.2', 'TD.1', 'Y/A', '1stD.2', 'Pen', 'Yds.3', '1stPy', '#Dr', 'Sc%', 'TO%', 'Start', 'Time', 'Plays', 'Yds.4', 'Pts']
OFF_RANK_FACTORS = ['PF', 'Yds', 'TO', 'FL', '1stD', 'Yds.1', 'TD', 'Int', 'NY/A', 'Att.1', 'Yds.2', 'TD.1', 'Y/A', 'Sc%', 'TO%', 'Start', 'Time', 'Plays', 'Yds.4', 'Pts']
DEF_RANK_FACTORS = ['PF', 'Yds', 'TO', 'FL', '1stD', 'Yds.1', 'TD', 'Int', 'NY/A', 'Att.1', 'Yds.2', 'TD.1', 'Y/A', 'Sc%', 'TO%', 'Start', 'Time', 'Plays', 'Yds.4', 'Pts']
NEXT_DRAFT_ORDER = ['Jaguars', 'Jets', 'Dolphins', 'Falcons', 'Bengals', 'Eagles', 'Lions', 'Panthers', 'Broncos', 'Cowboys', 
                    'Giants', '49ers', 'Chargers', 'Vikings', 'Patriots', 'Cardinals', 'Raiders', 'Dolphins', 'Redskins', 'Bears', 
                    'Colts', 'Titans', 'Jets', 'Steelers', 'Jaguars', 'Browns', 'Ravens', 'Saints',
                    'Packers', 'Bills', 'Buccaneers', 'Chiefs']

In [3]:
# Recreate raw data from file so that empty values are not perceived as Null=NaN
raw = pd.read_csv('Raw Data For Last ' + str(YEARS) + ' Drafts.csv', encoding = 'utf-8')
raw

Position Drafted      Team  Draft Year  Pick Number  Team PF  Team Yds  \
0                 QB   Bengals        2020            1    279.0    5169.0   
1                 DE  Redskins        2020            2    266.0    4395.0   
2                 DB     Lions        2020            3    341.0    5549.0   
3                  T    Giants        2020            4    341.0    5416.0   
4                 QB  Dolphins        2020            5    306.0    4960.0   
..               ...       ...         ...          ...      ...       ...   
631               RB   Vikings        2001           27    397.0    5961.0   
632               DB   Raiders        2001           28    479.0    5776.0   
633               DE      Rams        2001           29    540.0    7075.0   
634               WR     Colts        2001           30    429.0    6141.0   
635               TE    Ravens        2001           31    333.0    5014.0   

     Team Ply  Team Y/P  Team TO  Team FL  ...  Def. Rank Yds.3  \
0      1049.0       4.9     30.0     14.0  ...              NaN   
1       885.0       5.0     21.0      8.0  ...              NaN   
2      1021.0       5.4     23.0      8.0  ...              NaN   
3      1012.0       5.4     33.0     16.0  ...              NaN   
4      1022.0       4.9     26.0      8.0  ...              NaN   
..        ...       ...      ...      ...  ...              ...   
631     958.0       6.2     28.0     10.0  ...              NaN   
632    1023.0       5.6     20.0      9.0  ...              NaN   
633    1014.0       7.0     35.0     12.0  ...              NaN   
634    1026.0       6.0     29.0     14.0  ...              NaN   
635    1058.0       4.7     26.0      7.0  ...              NaN   

     Def. Rank 1stPy  Def. Rank #Dr  Def. Rank Sc%  Def. Rank TO%  \
0                NaN            NaN            9.0           27.0   
1                NaN            NaN            4.0           15.0   
2                NaN            NaN            7.0           25.0   
3                NaN            NaN            8.0           29.0   
4                NaN            NaN            1.0           28.0   
..               ...            ...            ...            ...   
631              NaN            NaN            6.0           31.0   
632              NaN            NaN           24.0            7.0   
633              NaN            NaN            2.0           22.0   
634              NaN            NaN           12.0           23.0   
635              NaN            NaN           31.0            1.0   

     Def. Rank Start  Def. Rank Time  Def. Rank Plays  Def. Rank Yds.4  \
0               12.0            18.0             15.0             28.0   
1               31.0            31.0             29.0             29.0   
2                3.0            21.0             28.0             30.0   
3               25.0            19.0             17.0             20.0   
4               27.0            20.0             20.0             26.0   
..               ...             ...              ...              ...   
631              7.0            30.0             30.0             30.0   
632              1.0             5.0             15.0             18.0   
633             25.0            15.0             14.0             24.0   
634              3.0            25.0             28.0             25.0   
635              2.0             1.0              1.0              2.0   

     Def. Rank Pts  
0             22.0  
1             30.0  
2             24.0  
3             25.0  
4             32.0  
..             ...  
631           27.0  
632           11.0  
633           30.0  
634           19.0  
635            1.0  

[636 rows x 124 columns]

In [4]:
# Data Cleaning
def clean_raw_df(raw, save, keep_id, keep_position_drafted=True) -> 'DataFrame':
    
    # Convert non-Numerical data types to Numerical
    raw = _convert_string_data(raw, keep_position_drafted)
    
    # Keep only the columns of data that we want
    raw = _isolate_key_columns(raw)
    
    # Create an ID column for later reference
    raw = _add_id_column(raw)
    
    # Save a copy for reference to identify what team/position/year occurred for stats
    if save:
        _save_reference_df(raw)
    
    # Remove Identifying Columns
    if keep_id == False:
        raw = _remove_id_columns(raw)
    
    return raw

def _convert_string_data(df, keep) -> 'DataFrame':
    team_start_list = []
    opp_start_list = []
    for i in list(df['Team Start']):
        team_start_list.append(float(str(i)[4:].strip()))
    for i in list(df['Opp. Start']):
        opp_start_list.append(float(str(i)[4:].strip()))
    
    df['Team Start'] = team_start_list
    df['Opp. Start'] = opp_start_list
    
    team_time_list = []
    opp_time_list = []
    for i in list(df['Team Time']):
        team_time_list.append(float((int(str(i)[0])*60)+int(str(i)[2:])))
    for i in list(df['Opp. Time']):
        opp_time_list.append(float((int(str(i)[0])*60)+int(str(i)[2:])))
        
    df['Team Time'] = team_time_list
    df['Opp. Time'] = opp_time_list
    
    if keep:
        num_positions = []
        for i in list(df['Position Drafted']):
            num_positions.append(UNIT_DICT[i])

        df['Position Drafted'] = num_positions
    
    return df
    
def _isolate_key_columns(df):
    factors = []
    for i in TEAM_FACTORS:
        factors.append('Team ' + i)
    for i in OPP_FACTORS:
        factors.append('Opp. ' + i)
    for i in OFF_RANK_FACTORS:
        factors.append('Off. Rank ' + i)
    for i in DEF_RANK_FACTORS:
        factors.append('Def. Rank ' + i)
    df = df[BASIC_FACTORS+factors]
    return df

def _add_id_column(df) -> 'DataFrame':
    df = df.reset_index()
    df = df.rename(columns={'index':'ID'})
    return df
    
def _save_reference_df(df):
    reference_df = df.copy(deep=True)
    reference_df.to_csv('Clean Reference For Last ' + str(YEARS) + ' Drafts.csv', encoding = 'utf-8-sig', index=False)
    
def _remove_id_columns(df):
    df = df.drop(columns=['Pick Number','Team', 'Draft Year'])
    return df

In [5]:
# We will use this dataframe for our model since it only has Numerical values, 
# but we will not use the ID when calculating distance
clean = clean_raw_df(raw, save=True, keep_id=False)
clean

ID  Position Drafted  Team PF  Team Yds  Team Ply  Team Y/P  Team TO  \
0      0                 1    279.0    5169.0    1049.0       4.9     30.0   
1      1                 2    266.0    4395.0     885.0       5.0     21.0   
2      2                 2    341.0    5549.0    1021.0       5.4     23.0   
3      3                 1    341.0    5416.0    1012.0       5.4     33.0   
4      4                 1    306.0    4960.0    1022.0       4.9     26.0   
..   ...               ...      ...       ...       ...       ...      ...   
631  631                 1    397.0    5961.0     958.0       6.2     28.0   
632  632                 2    479.0    5776.0    1023.0       5.6     20.0   
633  633                 2    540.0    7075.0    1014.0       7.0     35.0   
634  634                 1    429.0    6141.0    1026.0       6.0     29.0   
635  635                 1    333.0    5014.0    1058.0       4.7     26.0   

     Team FL  Team 1stD  Team Cmp  ...  Def. Rank Yds.2  Def. Rank TD.1  \
0       14.0      312.0     356.0  ...             32.0            27.0   
1        8.0      248.0     298.0  ...             31.0            17.0   
2        8.0      313.0     344.0  ...             21.0            14.0   
3       16.0      311.0     376.0  ...             20.0            28.0   
4        8.0      315.0     371.0  ...             27.0            21.0   
..       ...        ...       ...  ...              ...             ...   
631     10.0      319.0     307.0  ...             15.0            25.0   
632      9.0      337.0     284.0  ...              5.0             5.0   
633     12.0      380.0     380.0  ...             13.0            27.0   
634     14.0      357.0     357.0  ...             25.0            16.0   
635      7.0      288.0     287.0  ...              1.0             1.0   

     Def. Rank Y/A  Def. Rank Sc%  Def. Rank TO%  Def. Rank Start  \
0             25.0            9.0           27.0             12.0   
1             26.0            4.0           15.0             31.0   
2             10.0            7.0           25.0              3.0   
3              4.0            8.0           29.0             25.0   
4             22.0            1.0           28.0             27.0   
..             ...            ...            ...              ...   
631           27.0            6.0           31.0              7.0   
632           14.0           24.0            7.0              1.0   
633           25.0            2.0           22.0             25.0   
634           23.0           12.0           23.0              3.0   
635            1.0           31.0            1.0              2.0   

     Def. Rank Time  Def. Rank Plays  Def. Rank Yds.4  Def. Rank Pts  
0              18.0             15.0             28.0           22.0  
1              31.0             29.0             29.0           30.0  
2              21.0             28.0             30.0           24.0  
3              19.0             17.0             20.0           25.0  
4              20.0             20.0             26.0           32.0  
..              ...              ...              ...            ...  
631            30.0             30.0             30.0           27.0  
632             5.0             15.0             18.0           11.0  
633            15.0             14.0             24.0           30.0  
634            25.0             28.0             25.0           19.0  
635             1.0              1.0              2.0            1.0  

[636 rows x 102 columns]

In [6]:
# We will use this dataframe to reference the Team/Year/Position Drafted for our test values and our test's closest neighbors
reference = pd.read_csv('Clean Reference For Last ' + str(YEARS) + ' Drafts.csv', encoding = 'utf-8')
reference

ID  Position Drafted      Team  Draft Year  Pick Number  Team PF  \
0      0                 1   Bengals        2020            1    279.0   
1      1                 2  Redskins        2020            2    266.0   
2      2                 2     Lions        2020            3    341.0   
3      3                 1    Giants        2020            4    341.0   
4      4                 1  Dolphins        2020            5    306.0   
..   ...               ...       ...         ...          ...      ...   
631  631                 1   Vikings        2001           27    397.0   
632  632                 2   Raiders        2001           28    479.0   
633  633                 2      Rams        2001           29    540.0   
634  634                 1     Colts        2001           30    429.0   
635  635                 1    Ravens        2001           31    333.0   

     Team Yds  Team Ply  Team Y/P  Team TO  ...  Def. Rank Yds.2  \
0      5169.0    1049.0       4.9     30.0  ...             32.0   
1      4395.0     885.0       5.0     21.0  ...             31.0   
2      5549.0    1021.0       5.4     23.0  ...             21.0   
3      5416.0    1012.0       5.4     33.0  ...             20.0   
4      4960.0    1022.0       4.9     26.0  ...             27.0   
..        ...       ...       ...      ...  ...              ...   
631    5961.0     958.0       6.2     28.0  ...             15.0   
632    5776.0    1023.0       5.6     20.0  ...              5.0   
633    7075.0    1014.0       7.0     35.0  ...             13.0   
634    6141.0    1026.0       6.0     29.0  ...             25.0   
635    5014.0    1058.0       4.7     26.0  ...              1.0   

     Def. Rank TD.1  Def. Rank Y/A  Def. Rank Sc%  Def. Rank TO%  \
0              27.0           25.0            9.0           27.0   
1              17.0           26.0            4.0           15.0   
2              14.0           10.0            7.0           25.0   
3              28.0            4.0            8.0           29.0   
4              21.0           22.0            1.0           28.0   
..              ...            ...            ...            ...   
631            25.0           27.0            6.0           31.0   
632             5.0           14.0           24.0            7.0   
633            27.0           25.0            2.0           22.0   
634            16.0           23.0           12.0           23.0   
635             1.0            1.0           31.0            1.0   

     Def. Rank Start  Def. Rank Time  Def. Rank Plays  Def. Rank Yds.4  \
0               12.0            18.0             15.0             28.0   
1               31.0            31.0             29.0             29.0   
2                3.0            21.0             28.0             30.0   
3               25.0            19.0             17.0             20.0   
4               27.0            20.0             20.0             26.0   
..               ...             ...              ...              ...   
631              7.0            30.0             30.0             30.0   
632              1.0             5.0             15.0             18.0   
633             25.0            15.0             14.0             24.0   
634              3.0            25.0             28.0             25.0   
635              2.0             1.0              1.0              2.0   

     Def. Rank Pts  
0             22.0  
1             30.0  
2             24.0  
3             25.0  
4             32.0  
..             ...  
631           27.0  
632           11.0  
633           30.0  
634           19.0  
635            1.0  

[636 rows x 105 columns]

In [7]:
# Split into Train / Test Dataframes to Test Model Accuracy on 2020 NFL Draft

def split_dfs_for_testing(final_df) -> list:

    test_df = final_df.iloc[:32]
    train_df = final_df.iloc[32:]
    print(train_df.shape, test_df.shape)

    combine = [train_df, test_df]
    return combine

In [8]:
combine = split_dfs_for_testing(clean)
train_df = combine[0]
test_df = combine[1] 
train_df = train_df.drop(columns='ID')
train_df

(604, 102) (32, 102)


Position Drafted  Team PF  Team Yds  Team Ply  Team Y/P  Team TO  \
32                  1    225.0    3865.0     902.0       4.3     28.0   
33                  2    342.0    5769.0    1003.0       5.8     32.0   
34                  2    333.0    4787.0     971.0       4.9     30.0   
35                  2    290.0    5379.0     995.0       5.4     24.0   
36                  2    396.0    6648.0    1055.0       6.3     35.0   
..                ...      ...       ...       ...       ...      ...   
631                 1    397.0    5961.0     958.0       6.2     28.0   
632                 2    479.0    5776.0    1023.0       5.6     20.0   
633                 2    540.0    7075.0    1014.0       7.0     35.0   
634                 1    429.0    6141.0    1026.0       6.0     29.0   
635                 1    333.0    5014.0    1058.0       4.7     26.0   

     Team FL  Team 1stD  Team Cmp  Team Att  ...  Def. Rank Yds.2  \
32      10.0      239.0     283.0     495.0  ...             32.0   
33      12.0      344.0     331.0     532.0  ...             14.0   
34      11.0      258.0     299.0     524.0  ...             26.0   
35      14.0      300.0     382.0     556.0  ...             30.0   
36       9.0      388.0     408.0     625.0  ...             24.0   
..       ...        ...       ...       ...  ...              ...   
631     10.0      319.0     307.0     495.0  ...             15.0   
632      9.0      337.0     284.0     475.0  ...              5.0   
633     12.0      380.0     380.0     587.0  ...             13.0   
634     14.0      357.0     357.0     571.0  ...             25.0   
635      7.0      288.0     287.0     504.0  ...              1.0   

     Def. Rank TD.1  Def. Rank Y/A  Def. Rank Sc%  Def. Rank TO%  \
32             32.0           27.0           13.0           26.0   
33             15.0            7.0            8.0           32.0   
34             21.0           19.0            9.0           21.0   
35             21.0           22.0            2.0           23.0   
36             29.0           20.0            6.0           28.0   
..              ...            ...            ...            ...   
631            25.0           27.0            6.0           31.0   
632             5.0           14.0           24.0            7.0   
633            27.0           25.0            2.0           22.0   
634            16.0           23.0           12.0           23.0   
635             1.0            1.0           31.0            1.0   

     Def. Rank Start  Def. Rank Time  Def. Rank Plays  Def. Rank Yds.4  \
32              25.0            28.0             21.0             10.0   
33              30.0            16.0             19.0              9.0   
34              29.0             1.0              2.0              8.0   
35              27.0            18.0             17.0             29.0   
36              22.0            10.0              7.0             26.0   
..               ...             ...              ...              ...   
631              7.0            30.0             30.0             30.0   
632              1.0             5.0             15.0             18.0   
633             25.0            15.0             14.0             24.0   
634              3.0            25.0             28.0             25.0   
635              2.0             1.0              1.0              2.0   

     Def. Rank Pts  
32            22.0  
33            26.0  
34            19.0  
35            32.0  
36            29.0  
..             ...  
631           27.0  
632           11.0  
633           30.0  
634           19.0  
635            1.0  

[604 rows x 101 columns]

In [9]:
test_df = test_df.drop(columns='Position Drafted')
test_df

ID  Team PF  Team Yds  Team Ply  Team Y/P  Team TO  Team FL  Team 1stD  \
0    0    279.0    5169.0    1049.0       4.9     30.0     14.0      312.0   
1    1    266.0    4395.0     885.0       5.0     21.0      8.0      248.0   
2    2    341.0    5549.0    1021.0       5.4     23.0      8.0      313.0   
3    3    341.0    5416.0    1012.0       5.4     33.0     16.0      311.0   
4    4    306.0    4960.0    1022.0       4.9     26.0      8.0      315.0   
5    5    337.0    5879.0     997.0       5.9     31.0     11.0      349.0   
6    6    340.0    5469.0    1077.0       5.1     35.0     14.0      335.0   
7    7    361.0    5467.0    1000.0       5.5     18.0      6.0      314.0   
8    8    300.0    5468.0    1020.0       5.4     20.0     12.0      298.0   
9    9    335.0    5455.0     973.0       5.6     28.0      7.0      305.0   
10  10    276.0    4368.0     956.0       4.6     25.0      9.0      253.0   
11  11    313.0    5819.0     989.0       5.9     17.0      9.0      315.0   
12  12    458.0    6366.0    1086.0       5.9     41.0     11.0      353.0   
13  13    479.0    6097.0    1012.0       6.0     23.0     10.0      336.0   
14  14    282.0    4777.0     954.0       5.0     16.0      6.0      279.0   
15  15    381.0    6075.0    1096.0       5.5     25.0     10.0      383.0   
16  16    434.0    6904.0    1069.0       6.5     18.0      7.0      379.0   
17  17    306.0    4960.0    1022.0       4.9     26.0      8.0      315.0   
18  18    313.0    5819.0     989.0       5.9     17.0      9.0      315.0   
19  19    300.0    5468.0    1020.0       5.4     20.0     12.0      298.0   
20  20    385.0    5772.0    1104.0       5.2     23.0     15.0      354.0   
21  21    407.0    5656.0     970.0       5.8     20.0     12.0      314.0   
22  22    337.0    5879.0     997.0       5.9     31.0     11.0      349.0   
23  23    458.0    5982.0    1011.0       5.9      8.0      2.0      347.0   
24  24    479.0    6097.0    1012.0       6.0     23.0     10.0      336.0   
25  25    376.0    5528.0    1020.0       5.4     13.0      9.0      320.0   
26  26    405.0    5991.0    1046.0       5.7     20.0     14.0      341.0   
27  27    531.0    6521.0    1064.0       6.1     15.0      7.0      386.0   
28  28    402.0    5805.0     949.0       6.1     17.0      9.0      317.0   
29  29    306.0    4960.0    1022.0       4.9     26.0      8.0      315.0   
30  30    407.0    5656.0     970.0       5.8     20.0     12.0      314.0   
31  31    451.0    6067.0     976.0       6.2     15.0     10.0      350.0   

    Team Cmp  Team Att  ...  Def. Rank Yds.2  Def. Rank TD.1  Def. Rank Y/A  \
0      356.0     616.0  ...             32.0            27.0           25.0   
1      298.0     479.0  ...             31.0            17.0           26.0   
2      344.0     571.0  ...             21.0            14.0           10.0   
3      376.0     607.0  ...             20.0            28.0            4.0   
4      371.0     615.0  ...             27.0            21.0           22.0   
5      394.0     597.0  ...             18.0            21.0           15.0   
6      382.0     633.0  ...             29.0            32.0           32.0   
7      355.0     554.0  ...             24.0             5.0           20.0   
8      364.0     589.0  ...             28.0            31.0           31.0   
9      318.0     539.0  ...             30.0            28.0           30.0   
10     323.0     521.0  ...              2.0             9.0            2.0   
11     367.0     523.0  ...              8.0            21.0            5.0   
12     382.0     630.0  ...              1.0             7.0            1.0   
13     331.0     478.0  ...             17.0             7.0           23.0   
14     312.0     504.0  ...             16.0             5.0           13.0   
15     459.0     684.0  ...             15.0            14.0           16.0   
16     388.0     597.0  ...             11.0            17.0            8.0   
17     

In [10]:
def convert_df_to_weighted_df(df):
    std_list = list(df.std(axis = 0, skipna= True))
    mean_list = list(df.mean(axis=0, skipna= True))
    frames = [pd.DataFrame(df.loc[df.index[0]], columns=df.columns)]
    for i in range(df.index[0], df.index[0]+len(df.index)):
        row_list = list(df.loc[i])
        weighted_row = []
        for j in range(len(row_list)):
            if j == 0:
                weighted_row.append(row_list[j])
            else:
                weighted_row.append((row_list[j] - mean_list[j])/std_list[j])
        weighted_df_row = pd.DataFrame([weighted_row], columns=df.columns)
        frames.append(weighted_df_row)
    weighted_df = pd.concat(frames)
    weighted_df.index = df.index
    return weighted_df

In [11]:
train_df = convert_df_to_weighted_df(train_df)
test_df = convert_df_to_weighted_df(test_df)
train_df

Position Drafted   Team PF  Team Yds  Team Ply  Team Y/P   Team TO  \
32                1.0 -1.647166 -2.215272 -2.272113 -1.777888  0.214048   
33                2.0 -0.011434  0.708710 -0.136331  1.032242  0.793800   
34                2.0 -0.137260 -0.799352 -0.813014 -0.653836  0.503924   
35                2.0 -0.738426  0.109785 -0.305502  0.282874 -0.365705   
36                2.0  0.743519  2.058595  0.963280  1.968952  1.228614   
..                ...       ...       ...       ...       ...       ...   
631               1.0  0.757499  1.003565 -1.087917  1.781610  0.214048   
632               2.0  1.903909  0.719460  0.286596  0.657558 -0.945457   
633               2.0  2.756726  2.714341  0.096279  3.280346  1.228614   
634               1.0  1.204879  1.279992  0.350035  1.406926  0.358986   
635               1.0 -0.137260 -0.450747  1.026719 -1.028520 -0.075829   

      Team FL  Team 1stD  Team Cmp  Team Att  ...  Def. Rank Yds.2  \
32  -0.211270  -1.697137 -0.940876 -0.773119  ...         1.612923   
33   0.327157   1.061820  0.045059 -0.144795  ...        -0.349882   
34   0.057943  -1.197897 -0.612231 -0.280649  ...         0.958654   
35   0.865583  -0.094314  1.092616  0.262767  ...         1.394833   
36  -0.480483   2.217954  1.626664  1.434508  ...         0.740565   
..        ...        ...       ...       ...  ...              ...   
631 -0.211270   0.404925 -0.447908 -0.773119  ...        -0.240837   
632 -0.480483   0.877889 -0.920336 -1.112754  ...        -1.331284   
633  0.327157   2.007747  1.051535  0.789201  ...        -0.458926   
634  0.865583   1.403405  0.579108  0.517493  ...         0.849610   
635 -1.018910  -0.409624 -0.858715 -0.620284  ...        -1.767463   

     Def. Rank TD.1  Def. Rank Y/A  Def. Rank Sc%  Def. Rank TO%  \
32         1.666576       1.113280      -0.258618       0.978279   
33        -0.168731      -1.083457      -0.798373       1.636153   
34         0.479024       0.234585      -0.690422       0.430051   
35         0.479024       0.564096      -1.446079       0.649342   
36         1.342698       0.344422      -1.014275       1.197570   
..              ...            ...            ...            ...   
631        0.910861       1.113280      -1.014275       1.526507   
632       -1.248323      -0.314599       0.928844      -1.104989   
633        1.126780       0.893606      -1.446079       0.539696   
634       -0.060772       0.673933      -0.366569       0.649342   
635       -1.680160      -1.742478       1.684501      -1.762862   

     Def. Rank Start  Def. Rank Time  Def. Rank Plays  Def. Rank Yds.4  \
32          0.883752        1.210413         0.416338        -0.793731   
33          1.430105       -0.091810         0.198333        -0.902098   
34          1.320834       -1.719587        -1.654703        -1.010464   
35          1.102293        0.125227        -0.019671         1.265233   
36          0.555941       -0.742921        -1.109692         0.940134   
..               ...             ...              ...              ...   
631        -1.083117        1.427450         1.397357         1.373600   
632        -1.738739       -1.285513        -0.237675         0.073201   
633         0.883752       -0.200328        -0.346677         0.723400   
634        -1.520198        0.884857         1.179352         0.831767   
635        -1.629469       -1.719587        -1.763705        -1.660663   

     Def. Rank Pts  
32        0.485588  
33        0.918176  
34        0.161146  
35        1.567059  
36        1.242618  
..             ...  
631       1.026324  
632      -0.704031  
633       1.350765  
634       0.161146  
635      -1.785502  

[604 rows x 101 columns]

In [12]:
X_train = train_df.drop("Position Drafted", axis=1)
Y_train = train_df["Position Drafted"]
X_test  = test_df.drop("ID", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((604, 100), (604,), (32, 100))

In [14]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

67.72

In [15]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False).head(10)

Feature  Correlation
44       Opp. Att.1     1.177419
38         Opp. Att     0.772528
47         Opp. Y/A     0.597795
26        Team Time     0.562363
10          Team TD     0.554151
22         Team #Dr     0.532513
66     Off. Rank TD     0.402271
98  Def. Rank Yds.4     0.379295
84   Def. Rank 1stD     0.329531
88   Def. Rank NY/A     0.292431

In [16]:
coeff_df.sort_values(by='Correlation', ascending=False).tail(10)

Feature  Correlation
52         Opp. #Dr    -0.347176
70  Off. Rank Yds.2    -0.361266
23         Team Sc%    -0.362191
48      Opp. 1stD.2    -0.394779
97  Def. Rank Plays    -0.434750
92    Def. Rank Y/A    -0.435089
30          Opp. PF    -0.468484
89  Def. Rank Att.1    -0.564344
74    Off. Rank TO%    -0.639125
62     Off. Rank TO    -0.721202

In [17]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

74.5

In [18]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

73.68

In [19]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

60.6

In [20]:
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

55.3

In [21]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

c:\users\ethan\desktop\football-data-science\env\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


68.71

In [22]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

62.25

In [23]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

93.87

In [24]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

93.87

In [25]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

Model  Score
3               Random Forest  93.87
8               Decision Tree  93.87
0     Support Vector Machines  74.50
1                         KNN  73.68
7                  Linear SVC  68.71
2         Logistic Regression  67.72
6  Stochastic Gradient Decent  62.25
4                 Naive Bayes  60.60
5                  Perceptron  55.30

In [ ]:
# Y_pred = decision_tree.predict(X_test)

In [26]:
submission = pd.DataFrame({
        "Prediction": Y_pred
        # "X_Test": X_test
    })
submission = pd.concat([submission, reference.iloc[:32]], axis=1)
submission

Prediction  ID  Position Drafted        Team  Draft Year  Pick Number  \
0          1.0   0                 1     Bengals        2020            1   
1          2.0   1                 2    Redskins        2020            2   
2          1.0   2                 2       Lions        2020            3   
3          1.0   3                 1      Giants        2020            4   
4          1.0   4                 1    Dolphins        2020            5   
5          1.0   5                 1    Chargers        2020            6   
6          1.0   6                 2    Panthers        2020            7   
7          2.0   7                 2   Cardinals        2020            8   
8          2.0   8                 2     Jaguars        2020            9   
9          2.0   9                 1      Browns        2020           10   
10         1.0  10                 1        Jets        2020           11   
11         2.0  11                 1     Raiders        2020           12   
12         2.0  12                 1  Buccaneers        2020           13   
13         2.0  13                 2       49ers        2020           14   
14         1.0  14                 1     Broncos        2020           15   
15         2.0  15                 2     Falcons        2020           16   
16         2.0  16                 1     Cowboys        2020           17   
17         1.0  17                 1    Dolphins        2020           18   
18         2.0  18                 2     Raiders        2020           19   
19         2.0  19                 2     Jaguars        2020           20   
20         2.0  20                 1      Eagles        2020           21   
21         2.0  21                 1     Vikings        2020           22   
22         1.0  22                 2    Chargers        2020           23   
23         2.0  23                 1      Saints        2020           24   
24         2.0  24                 1       49ers        2020           25   
25         2.0  25                 1     Packers        2020           26   
26         2.0  26                 2    Seahawks        2020           27   
27         1.0  27                 2      Ravens        2020           28   
28         2.0  28                 1      Titans        2020           29   
29         1.0  29                 2    Dolphins        2020           30   
30         2.0  30                 2     Vikings        2020           31   
31         2.0  31                 1      Chiefs        2020           32   

    Team PF  Team Yds  Team Ply  Team Y/P  ...  Def. Rank Yds.2  \
0     279.0    5169.0    1049.0       4.9  ...             32.0   
1     266.0    4395.0     885.0       5.0  ...             31.0   
2     341.0    5549.0    1021.0       5.4  ...             21.0   
3     341.0    5416.0    1012.0       5.4  ...             20.0   
4     306.0    4960.0    1022.0       4.9  ...             27.0   
5     337.0    5879.0     997.0       5.9  ...             18.0   
6     340.0    5469.0    1077.0       5.1  ...             29.0   
7     361.0    5467.0    1000.0       5.5  ...             24.0   
8     300.0    5468.0    1020.0       5.4  ...             28.0   
9     335.0    5455.0     973.0       5.6  ...             30.0   
10    276.0    4368.0     956.0       4.6  ...              2.0   
11    313.0    5819.0     989.0       5.9  ...              8.0   
12    458.0    6366.0    1086.0       5.9  ...              1.0   
13    479.0    6097.0    1012.0       6.0  ...             17.0   
14    282.0    4777.0     954.0       5.0  ...             16.0   
15    381.0    6075.0    1096.0       5.5  ...             15.0   
16    434.0    6904.0    1069.0       6.5  ...             11.0   
17    306.0    4960.0    1022.0       4.9  ...             27.0   
18    313.0    5819.0     989.0       5.9  ...              8.0   
19    300.0    5468.0    1020.0       5.4  ...             28.0   
20    385.0    5772.0    1104.0       5.2  ...         

In [27]:
def test_unit_prediction(submission):
    correct_predictions = 0
    top_10 = 0
    mid_10 = 0
    last_12 = 0
    
    prediction_list = list(submission['Prediction'])
    actual_list = list(submission['Position Drafted'])
    length = len(prediction_list)
    
    for i in range(length):
        if prediction_list[i] == actual_list[i]:
            correct_predictions += 1
            if i <= 9:
                top_10 += 1
            elif i <= 19:
                mid_10 += 1
            elif i <= 31:
                last_12 += 1
    
    correct_pred_percent = round(correct_predictions/length, 4)*100
    top_10_accuracy = round(top_10 / 10, 4)*100
    mid_10_accuracy = round(mid_10 / 10, 4)*100
    last_12_accuracy = round(last_12 / 12, 4)*100
    
    print()
    print('Correct Predictions %: ', correct_pred_percent)
    print('Correct Top 10 Predictions %: ', top_10_accuracy)
    print('Correct Mid 10 Predictions %: ', mid_10_accuracy)
    print('Correct Last 12 Predictions %: ', last_12_accuracy)
    print()
    
    return (correct_pred_percent, top_10_accuracy, mid_10_accuracy, last_12_accuracy)


In [28]:
test_unit_prediction(submission)


Correct Predictions %:  50.0
Correct Top 10 Predictions %:  70.0
Correct Mid 10 Predictions %:  70.0
Correct Last 12 Predictions %:  16.669999999999998



(50.0, 70.0, 70.0, 16.669999999999998)

In [29]:
reference2 = pd.read_csv('Raw Data For ' + str(CURRENT_YEAR+1) + ' Draft.csv', encoding = 'utf-8')
reference2 = clean_raw_df(reference2, save=False, keep_id =True, keep_position_drafted=False)
reference2.drop(columns='Position Drafted', inplace=True)
reference2

ID        Team  Draft Year  Pick Number  Team PF  Team Yds  Team Ply  \
0    0     Jaguars        2021            1    292.0    4935.0     932.0   
1    1        Jets        2021            2    229.0    4129.0     886.0   
2    2    Dolphins        2021            3    378.0    4970.0     941.0   
3    3     Falcons        2021            4    369.0    5510.0    1007.0   
4    4     Bengals        2021            5    308.0    4921.0    1000.0   
5    5      Eagles        2021            6    320.0    5138.0    1001.0   
6    6       Lions        2021            7    342.0    5186.0     937.0   
7    7      Giants        2021            8    257.0    4458.0     911.0   
8    8    Panthers        2021            9    343.0    5272.0     933.0   
9    9     Broncos        2021           10    292.0    4923.0     955.0   
10  10     Cowboys        2021           11    376.0    5642.0    1033.0   
11  11    Chargers        2021           12    346.0    5697.0    1058.0   
12  12     Vikings        2021           13    393.0    5784.0     949.0   
13  13    Patriots        2021           14    298.0    4832.0     916.0   
14  14       49ers        2021           15    353.0    5594.0     982.0   
15  15     Raiders        2021           16    402.0    5668.0     972.0   
16  16   Cardinals        2021           17    403.0    5939.0    1034.0   
17  17       Colts        2021           18    423.0    5612.0     968.0   
18  18    Redskins        2021           19    315.0    4828.0     989.0   
19  19       Bears        2021           20    356.0    4946.0     969.0   
20  20        Rams        2021           21    354.0    5699.0    1018.0   
21  21      Browns        2021           22    384.0    5555.0     960.0   
22  22    Dolphins        2021           23    378.0    4970.0     941.0   
23  23      Ravens        2021           24    430.0    5285.0     920.0   
24  24      Titans        2021           25    450.0    5851.0     961.0   
25  25  Buccaneers        2021           26    448.0    5660.0     953.0   
26  26        Jets        2021           27    229.0    4129.0     886.0   
27  27    Steelers        2021           28    394.0    4960.0     978.0   
28  28      Saints        2021           29    449.0    5676.0     981.0   
29  29       Bills        2021           30    445.0    5888.0     970.0   
30  30     Packers        2021           31    474.0    5908.0     946.0   
31  31      Chiefs        2021           32    452.0    6385.0    1007.0   

    Team Y/P  Team TO  Team FL  ...  Def. Rank Yds.2  Def. Rank TD.1  \
0        5.3     24.0      8.0  ...             30.0            30.0   
1        4.7     17.0      5.0  ...             10.0            16.0   
2        5.3     16.0      6.0  ...             17.0            14.0   
3        5.5     16.0      5.0  ...              7.0            13.0   
4        4.9     22.0     13.0  ...             25.0             4.0   
5        5.1     26.0      8.0  ...             24.0            27.0   
6        5.5     19.0      7.0  ...             29.0            32.0   
7        4.9     20.0     10.0  ...             11.0            11.0   
8        5.7     16.0      5.0  ...             16.0            16.0   
9        5.2     32.0      9.0  ...             26.0            27.0   
10       5.5     25.0     13.0  ...             32.0            25.0   
11       5.4     16.0      6.0  ...             21.0            16.0   
12       6.1     23.0     10.0  ...             28.0            22.0   
13       5.3     19.0      5.0  ...             27.0            16.0   
14       5.7     30.0     13.0  ...              6.0             4.0   
15       5.8     22.0     14.0  ...             22.0            31.0   
16       5.7     20.0      8.0  ...             23.0            11.0   
17       5.8     14.0      4.0  ...              2.0            16.0   
18       4.9     25.0     11.0  ...             12.0             7.0   
19       5.1     20.0      5.0  ...             15.0   

In [30]:
test_df_2 = reference2.drop(columns=['Team', 'Draft Year', 'Pick Number'])
test_df_2 = convert_df_to_weighted_df(test_df_2)
test_df_2

ID   Team PF  Team Yds  Team Ply  Team Y/P   Team TO   Team FL  \
0    0.0 -1.114695 -0.706427 -0.808300 -0.454606  0.955240  0.064635   
1    1.0 -2.075869 -2.220987 -1.920279 -1.818424 -0.461805 -0.969521   
2    2.0  0.197384 -0.640658 -0.590739 -0.454606 -0.664239 -0.624802   
3    3.0  0.060073  0.374060  1.004709  0.000000 -0.664239 -0.969521   
4    4.0 -0.870587 -0.732734  0.835495 -1.363818  0.550370  1.788227   
5    5.0 -0.687507 -0.324968  0.859668 -0.909212  1.360109  0.064635   
6    6.0 -0.351858 -0.234771 -0.687433  0.000000 -0.056935 -0.280084   
7    7.0 -1.648681 -1.602761 -1.315942 -1.363818  0.145500  0.754072   
8    8.0 -0.336602 -0.073168 -0.784126  0.454606 -0.664239 -0.969521   
9    9.0 -1.114695 -0.728976 -0.252310 -0.681909  2.574719  0.409353   
10  10.0  0.166871  0.622102  1.633219  0.000000  1.157674  1.788227   
11  11.0 -0.290831  0.725453  2.237555 -0.227303 -0.664239 -0.624802   
12  12.0  0.426235  0.888935 -0.397351  1.363818  0.752805  0.754072   
13  13.0 -1.023155 -0.899975 -1.195075 -0.454606 -0.056935 -0.969521   
14  14.0 -0.184034  0.531905  0.400373  0.454606  2.169849  1.788227   
15  15.0  0.563546  0.670959  0.158638  0.681909  0.550370  2.132946   
16  16.0  0.578802  1.180197  1.657392  0.454606  0.145500  0.064635   
17  17.0  0.883937  0.565729  0.061944  0.681909 -1.069109 -1.314239   
18  18.0 -0.763790 -0.907491  0.569587 -1.363818  1.157674  1.098790   
19  19.0 -0.138264 -0.685757  0.086118 -0.909212  0.145500 -0.969521   
20  20.0 -0.168778  0.729211  1.270617  0.227303  0.752805  0.754072   
21  21.0  0.288924  0.458620 -0.131443  0.681909 -0.664239  0.064635   
22  22.0  0.197384 -0.640658 -0.590739 -0.454606 -0.664239 -0.624802   
23  23.0  0.990734 -0.048739 -1.098381  0.454606 -0.461805 -0.280084   
24  24.0  1.295869  1.014835 -0.107270  1.363818 -1.676414 -1.314239   
25  25.0  1.265355  0.655926 -0.300657  0.909212 -0.664239 -0.969521   
26  26.0 -2.075869 -2.220987 -1.920279 -1.818424 -0.461805 -0.969521   
27  27.0  0.441492 -0.659449  0.303679 -0.909212 -0.461805 -0.280084   
28  28.0  1.280612  0.685991  0.376199  0.681909 -0.461805  0.409353   
29  29.0  1.219585  1.084362  0.110291  1.363818  0.145500  1.098790   
30  30.0  1.662030  1.121944 -0.469871  1.591121 -1.878849 -0.969521   
31  31.0  1.326382  2.018278  1.004709  1.818424 -0.866674  0.064635   

    Team 1stD  Team Cmp  Team Att  ...  Def. Rank Yds.2  Def. Rank TD.1  \
0   -1.006668  0.462338  0.863772  ...         1.558015        1.669720   
1   -2.419817 -1.711978 -1.098988  ...        -0.703050        0.127381   
2   -0.171625 -0.175777 -0.468744  ...         0.088323       -0.092953   
3    0.567066  0.887747  1.043841  ...        -1.042210       -0.203121   
4   -0.492796  0.580507  0.611674  ...         0.992749       -1.194624   
5   -0.107391 -0.483017  0.701709  ...         0.879696        1.339218   
6    0.181662  0.296900  0.449612  ...         1.444962        1.890054   
7   -1.552658 -0.884793 -0.612800  ...        -0.589997       -0.423455   
8   -0.171625  0.320534 -0.234653  ...        -0.024730        0.127381   
9   -1.263604 -1.168399 -0.198639  ...         1.105802        1.339218   
10   0.695534  1.005917  1.187897  ...         1.784121        1.118884   
11   0.663417  1.171354  1.259925  ...         0.540536        0.127381   
12   0.920354 -0.483017 -0.900911  ...         1.331908        0.788383   
13  -0.460679 -1.901049 -2.107378  ...         1.218855        0.127381   
14   0.310130  0.107830  0.125486  ...        -1.155263       -1.194624   
15   0.438598  0.084196 -0.234653  ...         0.653589        1.779887   
16   1.369992  0.627775  0.395591  ...         0.766642       -0.423455   
17   0.567066  0.296900 -0.018570  ...        -1.607476        0.127381   
18  -0.557030  0.604141  0.773737  ...        -0.476943       -0.864123   
19  -0.300093  0.651409  0.827758  ...        -0.137784       -1.414958   
20   0.245896  0.675042  0.467618  ...        -1.49442

In [39]:
train_df_2 = pd.concat([test_df, train_df])
train_df_2 = train_df_2.drop(columns='ID')
listy = list(train_df_2['Position Drafted'])
listy2 = list(clean['Position Drafted'][:32])
train_df_2['Position Drafted'] = listy2+listy[32:]
train_df_2

Team PF  Team Yds  Team Ply  Team Y/P   Team TO   Team FL  Team 1stD  \
0   -1.238250 -0.796147  0.781036 -1.388433  1.056882  1.406268  -0.380443   
1   -1.424615 -2.166853 -2.698486 -1.175850 -0.232872 -0.608683  -2.404204   
2   -0.349434 -0.123191  0.186971 -0.325518  0.053740 -0.608683  -0.348822   
3   -0.349434 -0.358726 -0.003978 -0.325518  1.486799  2.077918  -0.412065   
4   -0.851185 -1.166273  0.208188 -1.388433  0.483658 -0.608683  -0.285580   
..        ...       ...       ...       ...       ...       ...        ...   
631  0.757499  1.003565 -1.087917  1.781610  0.214048 -0.211270   0.404925   
632  1.903909  0.719460  0.286596  0.657558 -0.945457 -0.480483   0.877889   
633  2.756726  2.714341  0.096279  3.280346  1.228614  0.327157   2.007747   
634  1.204879  1.279992  0.350035  1.406926  0.358986  0.865583   1.403405   
635 -0.137260 -0.450747  1.026719 -1.028520 -0.075829 -1.018910  -0.409624   

     Team Cmp  Team Att  Team Yds.1  ...  Def. Rank TD.1  Def. Rank Y/A  \
0   -0.028530  0.928962   -0.369800  ...        1.106330       0.761136   
1   -1.585913 -1.239605   -2.138523  ...       -0.017561       0.869870   
2   -0.350747  0.216659    0.152394  ...       -0.354728      -0.869870   
3    0.508499  0.786501   -0.203456  ...        1.218719      -1.522273   
4    0.374242  0.913133   -0.049745  ...        0.431996       0.434935   
..        ...       ...         ...  ...             ...            ...   
631 -0.447908 -0.773119    0.498524  ...        0.910861       1.113280   
632 -0.920336 -1.112754   -0.327701  ...       -1.248323      -0.314599   
633  1.051535  0.789201    2.697602  ...        1.126780       0.893606   
634  0.579108  0.517493    1.205371  ...       -0.060772       0.673933   
635 -0.858715 -0.620284   -1.098949  ...       -1.680160      -1.742478   

     Def. Rank Sc%  Def. Rank TO%  Def. Rank Start  Def. Rank Time  \
0        -0.556763       0.805546        -0.551625        0.095755   
1        -1.117027      -0.412287         1.612168        1.469349   
2        -0.780869       0.602574        -1.576579        0.412738   
3        -0.668816       1.008518         0.928865        0.201416   
4        -1.453186       0.907032         1.156633        0.307077   
..             ...            ...              ...             ...   
631      -1.014275       1.526507        -1.083117        1.427450   
632       0.928844      -1.104989        -1.738739       -1.285513   
633      -1.446079       0.539696         0.883752       -0.200328   
634      -0.366569       0.649342        -1.520198        0.884857   
635       1.684501      -1.762862        -1.629469       -1.719587   

     Def. Rank Plays  Def. Rank Yds.4  Def. Rank Pts  Position Drafted  
0          -0.289075         1.146995       0.334064               1.0  
1           1.271235         1.264635       1.197907               2.0  
2           1.159784         1.382276       0.550025               2.0  
3          -0.066174         0.205871       0.658005               1.0  
4           0.268178         0.911714       1.413867               1.0  
..               ...              ...            ...               ...  
631         1.397357         1.373600       1.026324               1.0  
632        -0.237675         0.073201      -0.704031               2.0  
633        -0.346677         0.723400       1.350765               2.0  
634         1.179352         0.831767       0.161146               1.0  
635        -1.763705        -1.660663      -1.785502               1.0  

[636 rows x 101 columns]

In [40]:
X_train = train_df_2.drop("Position Drafted", axis=1)
Y_train = train_df_2["Position Drafted"]
X_test  = test_df_2.drop("ID", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((636, 100), (636,), (32, 100))

In [41]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)

models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

c:\users\ethan\desktop\football-data-science\env\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Model  Score
3               Random Forest  93.40
8               Decision Tree  93.40
0     Support Vector Machines  74.37
1                         KNN  73.43
7                  Linear SVC  70.44
2         Logistic Regression  67.92
4                 Naive Bayes  59.43
5                  Perceptron  57.70
6  Stochastic Gradient Decent  53.93

In [43]:
submission = pd.DataFrame({
        "Prediction": Y_pred
        # "X_Test": X_test
    })
submission = pd.concat([submission, reference2], axis=1)
submission

Prediction  ID        Team  Draft Year  Pick Number  Team PF  Team Yds  \
0          1.0   0     Jaguars        2021            1    292.0    4935.0   
1          1.0   1        Jets        2021            2    229.0    4129.0   
2          1.0   2    Dolphins        2021            3    378.0    4970.0   
3          2.0   3     Falcons        2021            4    369.0    5510.0   
4          1.0   4     Bengals        2021            5    308.0    4921.0   
5          2.0   5      Eagles        2021            6    320.0    5138.0   
6          1.0   6       Lions        2021            7    342.0    5186.0   
7          1.0   7      Giants        2021            8    257.0    4458.0   
8          2.0   8    Panthers        2021            9    343.0    5272.0   
9          1.0   9     Broncos        2021           10    292.0    4923.0   
10         1.0  10     Cowboys        2021           11    376.0    5642.0   
11         2.0  11    Chargers        2021           12    346.0    5697.0   
12         2.0  12     Vikings        2021           13    393.0    5784.0   
13         1.0  13    Patriots        2021           14    298.0    4832.0   
14         2.0  14       49ers        2021           15    353.0    5594.0   
15         2.0  15     Raiders        2021           16    402.0    5668.0   
16         2.0  16   Cardinals        2021           17    403.0    5939.0   
17         2.0  17       Colts        2021           18    423.0    5612.0   
18         1.0  18    Redskins        2021           19    315.0    4828.0   
19         1.0  19       Bears        2021           20    356.0    4946.0   
20         2.0  20        Rams        2021           21    354.0    5699.0   
21         2.0  21      Browns        2021           22    384.0    5555.0   
22         1.0  22    Dolphins        2021           23    378.0    4970.0   
23         2.0  23      Ravens        2021           24    430.0    5285.0   
24         2.0  24      Titans        2021           25    450.0    5851.0   
25         2.0  25  Buccaneers        2021           26    448.0    5660.0   
26         1.0  26        Jets        2021           27    229.0    4129.0   
27         1.0  27    Steelers        2021           28    394.0    4960.0   
28         2.0  28      Saints        2021           29    449.0    5676.0   
29         2.0  29       Bills        2021           30    445.0    5888.0   
30         2.0  30     Packers        2021           31    474.0    5908.0   
31         2.0  31      Chiefs        2021           32    452.0    6385.0   

    Team Ply  Team Y/P  Team TO  ...  Def. Rank Yds.2  Def. Rank TD.1  \
0      932.0       5.3     24.0  ...             30.0            30.0   
1      886.0       4.7     17.0  ...             10.0            16.0   
2      941.0       5.3     16.0  ...             17.0            14.0   
3     1007.0       5.5     16.0  ...              7.0            13.0   
4     1000.0       4.9     22.0  ...             25.0             4.0   
5     1001.0       5.1     26.0  ...             24.0            27.0   
6      937.0       5.5     19.0  ...             29.0            32.0   
7      911.0       4.9     20.0  ...             11.0            11.0   
8      933.0       5.7     16.0  ...             16.0            16.0   
9      955.0       5.2     32.0  ...             26.0            27.0   
10    1033.0       5.5     25.0  ...             32.0            25.0   
11    1058.0       5.4     16.0  ...             21.0            16.0   
12     949.0       6.1     23.0  ...             28.0            22.0   
13     916.0       5.3     19.0  ...             27.0            16.0   
14     982.0       5.7     30.0  ...              6.0             4.0   
15     972.0       5.8     22.0  ...             22.0            31.0   
16    1034.0       5.7     20.0  ...             23.0            11.0   
17     968.0       5.8     14.0  ...              2.0            16.0   
18     989.0       4.9     25.0  ...      